# my-fair-bnb

Branch-and-bound method is a general algorithm to solve optimization problems, which allow access to a sub-optimality oracle -- a procedure, which, given an instance of a problem $
    \min\{f(x) \colon x \in X\}
$, returns a non-trivial lower bound $\tau$ such that $\tau \leq f(x)$ for all $x \in X$. BnB cuts-off subsets of the feasibility set $X$ in a methodical and structured manner, but still has to make sure no region is missed.
- In fact, bnb can be adapted to the case, when the problem allows access only to a SAT oracle, which, given $(f, X, \tau)$, decides whether $\exists x \in X$ such that $f(x) \leq \tau$.

In [ ]:
import numpy as np
import networkx as nx
from scipy import sparse as sp

from numpy import ndarray
from numpy.random import default_rng

from tqdm import tqdm
from matplotlib import pyplot as plt

A proc to draw value paths in the toy bnb tree

In [ ]:
from toybnb.tree import build_optresult


def path_to_root(T: nx.DiGraph, node: int) -> list[int, ...]:
    """Ascent path to the root form the node."""
    path = [node]
    while T.pred[node]:
        node = next(iter(T.pred[node]))
        path.append(node)

    return path


def path_to_best(T: nx.DiGraph) -> list[int, ...]:
    """Descend from the root down the tree to the best solution."""
    node = T.graph["root"]
    best = nx.get_node_attributes(T, "best")
    path, inc = [node], T.graph["incumbent"]
    while T[node]:
        for child in T[node]:
            if best.get(child) is inc:
                node = child
                break
        else:
            break
        path.append(node)

    return path


def get_value_paths(T: nx.DiGraph) -> dict[int, list]:
    """Extract the value function from the tree."""
    best = nx.get_node_attributes(T, "best")
    leaves = [n for n in T if not T[n]]

    vfn, nil = {}, build_optresult()
    for leaf in leaves:
        vf = vfn[leaf] = []
        path = path_to_root(T, leaf)
        for j, n in enumerate(reversed(path)):
            b = best.get(n, nil) or nil
            vf.append((j, b.fun))

    return vfn

Codes for different states of the search tree nodes.

In [ ]:
from toybnb.tree import Status

Map the status of a node to common color and nodesize.

In [ ]:
def default_color(code: Status) -> str:
    match code:
        case Status.FEASIBLE:
            return "C3"
        case Status.INFEASIBLE:
            return "C2"
        case Status.OPEN:
            return "fuchsia"
        case Status.PRUNED:
            return "C5"
        case Status.CLOSED:
            return "#0008"
        case Status.FATHOMED:
            return "C4"
        case _:  # XXX "unofficial" status codes for dev
            return "C1"

A proc to draw toy bnb trees (or from scip Tracer)

In [ ]:
from matplotlib.collections import PatchCollection


def draw_tree(
    T: nx.DiGraph,
    pos: dict = None,
    ax: plt.Axes = None,
    nodelist: list = None,
) -> None:
    ax = ax if ax is None else plt.gca()

    st = nx.get_node_attributes(T, "status")
    best = nx.get_node_attributes(T, "best")

    # draw only open, feasible, or infeasible nodes only
    allowed = {Status.OPEN, Status.INFEASIBLE, Status.FEASIBLE, Status.FATHOMED}
    # INFEASIBLE, PRUNED, FEASIBLE -- fathomed node
    # CLOSED -- no branching options, OPEN -- node yet to be explored
    if nodelist is None:
        nodelist = set([n for n, s in st.items() if s in allowed])
        nodelist.update(path_to_best(T))

    nodelist = list(nodelist)

    # assign colors, highlighting the branch of the best solution
    nodecolor, inc = [], T.graph["incumbent"]
    for n in nodelist:
        if inc is best.get(n):
            nodecolor.append("C3")

        else:
            nodecolor.append(default_color(st[n]))

    # plot the branching edges as lightly as possible
    col = PatchCollection(
        nx.draw_networkx_edges(
            T,
            pos,
            ax=ax,
            width=0.5,
            node_size=0,
            arrows=None,
            arrowsize=0,
            arrowstyle="-",
            edge_color="#0004",
        )
    )
    col.set_zorder(-10)

    col = nx.draw_networkx_nodes(
        T,
        pos,
        ax=ax,
        node_size=1,
        node_shape="s",
        nodelist=nodelist,
        node_color=nodecolor,
    )
    col.set_zorder(10)

    ax.set_frame_on(False)

Gather relevant info about a node

In [ ]:
def signed_gap(pb: float, db: float) -> float:
    if db < 0 <= pb or pb < 0 <= db:
        return float("inf")

    return (pb - db) / min(abs(pb), abs(db))


def node_nfo(T: nx.DiGraph, nn: list[int]) -> dict:
    order = {n: j for j, n in enumerate(nn, 1)}

    st = nx.get_node_attributes(T, "status")

    def special(n: int) -> str:
        return getattr(st[n], "name", "SPECIAL")

    lp = nx.get_node_attributes(T, "lp")
    inc = T.graph["incumbent"]

    def nfo(n: int) -> str:
        return f"{order.get(n, '')} {special(n)} {signed_gap(inc.fun, lp[n].fun):.3%}"

    return {n: nfo(n) for n in T}

Plotly provides a better tree visualization, than matplotlib

In [ ]:
import plotly.graph_objects as go
from matplotlib.colors import to_hex


def draw_tree_plolty(
    T: nx.DiGraph,
    pos: dict[..., tuple[float, float]],
    nodelist: list = None,
    labels: dict = None,
) -> None:
    # prepare the edge line segments and create their trace
    xy = []
    for u, v in T.edges:
        xy.append(pos[u])
        xy.append(pos[v])
        xy.append((None, None))
    x, y = zip(*xy)
    edges = go.Scatter(
        x=x,
        y=y,
        mode="lines",
        line=dict(width=0.5, color="#888"),
        hoverinfo="none",
    )

    # next, deal with nodes
    st = nx.get_node_attributes(T, "status")
    best = nx.get_node_attributes(T, "best")

    # draw only open, feasible, or infeasible nodes only
    allowed = {Status.OPEN, Status.INFEASIBLE, Status.FEASIBLE, Status.FATHOMED}
    # INFEASIBLE, PRUNED, FEASIBLE -- fathomed node
    # CLOSED -- no branching options, OPEN -- node yet to be explored
    if nodelist is None:
        nodelist = set([n for n, s in st.items() if s in allowed])
        nodelist.update(path_to_best(T))

    nodelist = list(nodelist)
    if not nodelist:
        return go.Figure()

    # assign colors, highlighting the branch of the best solution
    nodecolor, inc = [], T.graph["incumbent"]
    for n in nodelist:
        if inc is best.get(n):
            nodecolor.append("C3")

        else:
            nodecolor.append(default_color(st[n]))

    # prepare the nodes
    x, y = zip(*(pos[n] for n in nodelist))
    nodes = go.Scatter(
        x=x,
        y=y,
        mode="markers",
        hoverinfo="text",
        marker=dict(line_width=0),
    )
    nodes.marker.size = [2 if st[n] == Status.CLOSED else 4 for n in nodelist]
    nodes.marker.color = list(map(to_hex, nodecolor))
    if labels is not None:
        nodes.text = list(map(labels.get, nodelist))

    layout = go.Layout(
        title="BnB search tree",
        titlefont_size=16,
        showlegend=False,
        hovermode="closest",
        margin=dict(b=20, l=5, r=5, t=40),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    )

    return go.Figure(data=(edges, nodes), layout=layout)

<br>

#### A generic random MILP

A Mixed Integer Linear Programs (MILP) is a linear program with integrality constraints. A MILP has the following generic form:

$$
\min\Bigl\{
    c^\top x
    \colon
    A x \leq b
    \,, x \in \bigl[l, u\bigr]
    \,,
    x \in \mathbb{Z}^m \times \mathbb{R}^{n-m}
\Bigr\}
    \,, $$

where $A \in \mathbb{R}^{r \times n}$, $b \in \mathbb{R}^r$, $
    l, u \in \mathbb{R}^n
$ with $l \leq u$ and $1 \leq m \leq n$.

In [ ]:
from toybnb.milp import MILP, generate

# feasibility checkers
# from toybnb.milp import is_feasible_box, is_feasible_int

#### A generator for MIS problems.

For a undirected graph $G = (V, E)$ the Maximum independent set problem is
$$
\begin{aligned}
    & \underset{x\in \{0, 1\}}{\text{maximize}}
      & & \sum_{v \in G} x_v
          \\
    & \text{subject to}
      & & \forall uv \in E
          \colon x_u + x_v \leq 1
          \,.
\end{aligned}
    $$

In [ ]:
from toybnb.milp import generate_mis_ba

Invert our bnb solver at the branch variable selection, while still using the depth-first node selector.

In [ ]:
from toybnb import search as bnb
from toybnb.coro import Coroutine


def inverted_search(p: MILP, nodesel: callable = bnb.nodesel_dfs) -> ...:
    """Inverted variable branching as generator"""

    def branchrule(T: nx.DiGraph, node: int) -> int:
        return co.co_yield((T, node))

    # def nodesel(G: nx.DiGraph, *reschedule: int) -> int:
    #     # yields must use a tag for use in a state machine
    #     return co.co_yield(("nodesel", G, *reschedule))

    args = p, nodesel, branchrule
    co = Coroutine(bnb.search, args=args)
    return iter(co)


def branching(p: MILP, nodesel: callable = bnb.nodesel_dfs) -> tuple[nx.DiGraph, int]:
    """Branching that allows each node to be visited only once"""
    it, visited, var = inverted_search(p, nodesel), set(), None
    try:
        while True:
            T, node = it.send(var)
            while node in visited:
                T, node = it.throw(IndexError)

            visited.add(node)
            var = yield T, node
            # assert it.gi_frame.f_locals["self"].co_is_suspended

    except StopIteration as e:
        return e.value, None


def send(it: iter, value: ...) -> ...:
    """Send a value to the generator and get a value from it in return"""
    try:
        return it.send(value), False

    except StopIteration as e:
        return e.value, True


class GeneratorEnv:
    """A wrapper to make generators into envs."""

    def reset(self, it: iter) -> ...:
        self.it = iter(it)
        return send(self.it, None)

    def step(self, act: ...) -> ...:
        return send(self.it, act)

<br>

### Reference solution

Generate a simple problem and solve it with SCIP to get the reference

In [ ]:
# it = generate(100, 50, 10, seed=1458)
# it = generate(150, 50, 15, seed=1458)
# it = generate(50, 50, 10, seed=1458)  # many infeasible nodes
# it = generate_mis_ba(210, seed=454)

# it = generate(1500, 1200, 5, seed=53912)
it = generate(50, 38, 10, seed=1458)  # very slow ub decay

p = next(it)  # x = it.gi_frame.f_locals["p"]

Convert to SCIP and solve

In [ ]:
from toybnb.scip import to_scip, get_result

m = to_scip(
    p,
    **{
        "display/verblevel": 0,
    }
)
m.optimize()

sol = get_result(m)

sol.fun

<br>

### ToyBNB tree

BnB for MILP is an exhaustive search which employs _a continuous relaxation of the MILP_ as the oracle in order to eliminate sub-regions of the feasibility set.
Let the integer feasibility set and its continuous relaxation be, respectively,
$$
\begin{align}
S
    &= \bigl\{
        x \in \mathbb{Z}^m \times \mathbb{R}^{n-m}
        \colon A x \leq b
        \,, x \in [l, u] 
    \bigr\}
    \,, \\
\breve{S}
    &= \bigl\{
        x \in \mathbb{R}^n
        \colon A x \leq b
        \,, x \in [l, u] 
    \bigr\}
    \,.
\end{align}
$$
The linear problem
$$
\breve{f}
    = \min_x \{
        c^\top x
        \colon x \in \breve{S}
    \}
    \,, $$
offers a _lower bound on the achievable value_ of the objective in $S$, i.e. every integer-feasible $x \in S$ necessarily has $
    \breve{f} \leq c^\top x
$.

In [ ]:
from toybnb.tree import OptimizeResult


def sub_problem(T: nx.DiGraph, n: int) -> tuple[MILP, OptimizeResult, ndarray]:
    """Get the sub-problem of the node."""
    dt = T.nodes[n]
    p, lp = dt["p"], dt.get("lp")

    # unpack the bitmask of the fractional vars in the node
    mask = np.unpackbits(dt["mask"], count=p.n, bitorder="little")
    return p, lp, mask

Solve the MILP, generated above, with random variable branching.

In [ ]:
with tqdm(ncols=70) as pb:
    rng = default_rng(671)
    env = GeneratorEnv()
    (T, node), fin = env.reset(branching(p))
    while not fin:
        pb.update(1)

        # pick a random index
        _, _, mask = sub_problem(T, node)
        var = rng.choice(np.flatnonzero(mask))

        # branch
        (T, node), fin = env.step(var)

assert node is None

Plot the primal and dual bound history

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 2), dpi=300)
nn_rng, pv_rng, dv = map(np.array, zip(*T.graph["track"]))
ax.plot(dv)
ax.plot(pv_rng)
ax.axhline(sol.fun, zorder=-10, c="k", alpha=0.25)

Draw the value paths

In [ ]:
vfn = get_value_paths(T)

fig, ax = plt.subplots(1, 1, figsize=(10, 4), dpi=120)
for leaf, vf in vfn.items():
    ax.plot(*zip(*vf), label=leaf, c="C0", alpha=0.24)

Draw the search tree

In [ ]:
T_rng = T
pos_rng = nx.nx_agraph.graphviz_layout(T_rng, prog="dot", args="")

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=300)
draw_tree(T_rng, pos_rng, ax)
fig.savefig("rand_tree.pdf")

In [ ]:
T.nodes[0]["best"]

Sanity check

In [ ]:
# we have incumbent >= lp ins `dual` and either closed
#  or integer-feasible
st = nx.get_node_attributes(T, "status")
assert all(st[n] in (Status.FEASIBLE, Status.CLOSED) for _, n in T.graph["duals"])

is_leaf = {n: not bool(T[n]) for n in T}
is_fathomed = {n: s != Status.OPEN for n, s in st.items()}
assert all(is_fathomed.values())

assert T.nodes[T.graph["root"]]["best"] is T.graph["incumbent"]

Compare with SCIP's solution

In [ ]:
print(
    T_rng.graph["incumbent"].fun,
    sol.fun,
    np.isclose(T_rng.graph["incumbent"].fun, sol.fun),
)
# there may be solution multiplicity
# assert np.allclose(T_rng.graph["incumbent"].x, sol.x)

A random branching strategy is absolutely oblivious to the nature and geometry of the underlying problem.

<br>

### lower-bound aware depth-first node selection

A node selection strategy is as important as the variable selection heuristic: the latter's goal is to achieve subset cutoff as soon as possible, while the fomer's is to dive into the nodes with the loosest lower bound possible so as to eat up the potential primal-dual margin in a sub-tree as fast as possible.

In [ ]:
def nodesel_dfs_lb(G: nx.DiGraph, *reschedule: int) -> int:
    """Prioritize the child with the worst lp lower bound."""
    stack, dt = G.graph["queue"], G.nodes

    def lower_bound(n: int) -> float:
        return dt[n]["lp"].fun

    # the nodes in `reschedule` may be anything, but CLOSED
    if reschedule:
        # always reschedule the first node, since the current impl
        #  of the bnb search allows revisits, and relies on nodesel
        #  to pick a node and an unsuccessful branchrule to mark
        #  it as closed. Futhermore, reschedule it FIRST, so as to
        #  make this nodesel a true DFS.
        n, *reschedule = reschedule
        if dt[n]["status"] == Status.OPEN:
            stack.append(n)

        # prioritize the children by their lp lower bound, but
        #  schedule only OPEN nodes. The less tight the lower
        #  bound, intuitively, the more margin there is for an
        #  integer feasible solution.
        for n in sorted(reschedule, key=lower_bound, reverse=True):
            if dt[n]["status"] == Status.OPEN:
                stack.append(n)

    while stack:
        n = stack.pop()
        if dt[n]["status"] == Status.OPEN:
            return n

    raise IndexError

Run random branching with a slightly smarter nodesel

In [ ]:
with tqdm(ncols=70) as pb:
    rng = default_rng(671)
    env = GeneratorEnv()
    (T, node), fin = env.reset(branching(p, nodesel_dfs_lb))
    while not fin:
        pb.update(1)

        # pick a random index
        _, _, mask = sub_problem(T, node)
        var = rng.choice(np.flatnonzero(mask))

        # branch
        (T, node), fin = env.step(var)

assert node is None

The trace and the value function

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 2), dpi=300)
nn_rng_dfs, pv_rng_dfs, dv = map(np.array, zip(*T.graph["track"]))
ax.plot(dv)
ax.plot(pv_rng_dfs)
ax.axhline(sol.fun, zorder=-10, c="k", alpha=0.25)

In [ ]:
vfn = get_value_paths(T)

fig, ax = plt.subplots(1, 1, figsize=(10, 4), dpi=120)
for leaf, vf in vfn.items():
    ax.plot(*zip(*vf), label=leaf, c="C0", alpha=0.24)

The search tree

In [ ]:
T_rng_dfs = T
pos_rng_dfs = nx.nx_agraph.graphviz_layout(T_rng_dfs, prog="dot", args="")

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=300)
draw_tree(T_rng_dfs, pos_rng_dfs, ax)
fig.savefig("rand_dfs_lb_tree.pdf")

<br>

### Strong branching

Strong branching selects a variable to split the MILP with based on exhaustive look-ahead for the most promising  lower bound (lp relaxation, dual bound), with the goal to cut off a half of the search space as quickly as possible.

In [ ]:
# wrapper for scipy's LP solver for the relaxed
#  problem w/o integrality
from toybnb.tree import lpsolve

Suppose, we happen to have a candidate (incumbent) $x_*$ with $f^* = c^\top x_*$, which is integer-feasible in the __original problem's domain__ of which $S$ is a proper subset, but with $x_* \notin S$.
Then $f^* < \breve{f}$ __certifies__ that there is __no integer-feasible__ $x$ in $
    \breve{S} \supseteq S
$ with a lower objective value than $x_*$. This means that $\breve{S}$, including the entirety of $S$, may be excluded from the search.

The lp-gains $\Delta_\pm$ are computed based on the left and right relaxation of the integer problem.

If $\breve{f} \leq f^*$ and $\breve{x} \notin S$, there is some $j=1..m$ such that $\breve{x}^j \notin \mathbb{Z}$. Since it would be impossible for any integer-feasible solution $x \in S$ to have $
    x^j \in \bigl(
        \lfloor \breve{x}^j \rfloor,
        \lceil \breve{x}^j \rceil
    \bigr)
$, it becomes reasonable to split the original feasibility set $S$ in two non-adjacent subsets:
$$
    \underbrace{
        \bigl\{
            % x \colon
            x^j \leq \lfloor \breve{x}^j \rfloor
        \bigr\}  % \times \mathbb{R}^{n-1}
    }_{R^j_-}
    \uplus
    \underbrace{
        \bigl\{
            % x \colon
            \lceil \breve{x}^j \rceil \geq x^j
        \bigr\}  % \times \mathbb{R}^{n-1}
    }_{R^j_+}
    \,.
$$
Note that it is sufficient to split the region $S$ with respect to one variable only, since every other split is guaranteed by integer-feasiblity __not__ to contain a feasible solution in the excluded region of the $j$-th variable.

In [ ]:
# partition the feasibility set on a given integer
#  variable with the specified threshold
from toybnb.tree import split

The sub-regions $
    S^j_\pm = S \cap R^j_\pm
$ bring about new lower bounds $
\breve{f}^j_\pm
    = \min_x \{
        c^\top x\colon x \in \breve{S}^j_\pm
    \}
$. The new sub-problems are obtained from the original by modifying the bounds of the $j$-th variable: $
    \bigl[l^j, \lfloor \breve{x}^j \rfloor\bigr]
$ and $
    \bigl[\lceil \breve{x}^j \rceil, u^j\bigr]
$. The amount of the objective value, by which each bound is tightened, is the _gain_:
$$
\Delta^j_\pm
    = \breve{f}^j_\pm - \breve{f}
    \geq 0
    \,. $$

In [ ]:
from toybnb.tree import OptimizeResult, split


def get_lp_gains(
    p: MILP, lp: OptimizeResult, mask: ndarray
) -> tuple[ndarray, ndarray, int]:
    """Compute LP branching gains for each candidate variable in the binary mask."""
    lpiter = 0

    # split by the variable, solve LPs, and record dual gains
    gains = np.full((p.n, 2), np.nan)
    for j in np.flatnonzero(mask):
        lp_lo, lp_up = map(lpsolve, split(p, j, lp.x[j]))
        gains[j] = lp_lo.fun - lp.fun, lp_up.fun - lp.fun
        lpiter += lp_lo.nit + lp_up.nit

    return gains.clip(0), lpiter

Scoring functions
* additive $
    s_j = \mu \max\{\Delta^j_-, \Delta^j_+\}
        + (1 - \mu) \min\{\Delta^j_-, \Delta^j_+\}
$
* multiplicative $
    s_j = \Delta^j_- \Delta^j_+
$ -- the default scorefunc in SCIP

The strong branching rule enumerates all fractional variables and evaluates the lp lower bound changes on a split with respect to each one.

In [ ]:
"""Choose a variable to split the problem with, based on exhaustive look-ahead."""

with tqdm(ncols=70) as pb:
    n_lpit = 0
    env = GeneratorEnv()
    (T, node), fin = env.reset(branching(p, nodesel_dfs_lb))
    while not fin:
        pb.update(1)

        # get the up-lo branching gains
        p_, lp_, mask = sub_problem(T, node)
        gains, nit = get_lp_gains(p_, lp_, mask)
        n_lpit += nit

        # scores = mu * gains.max(-1) + (1 - mu) * gains.min(-1)
        scores = gains[:, 0] * gains[:, 1]

        # branch
        var = np.nanargmax(scores)
        (T, node), fin = env.step(var)

assert node is None

Plot the track, the value paths, and the tree

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots(1, 1, figsize=(5, 2), dpi=300)
nn_sb, pv_sb, dv = map(np.array, zip(*T.graph["track"]))
ax.plot(dv)
ax.plot(pv_sb)
ax.axhline(sol.fun, zorder=-10, c="k", alpha=0.25)

In [ ]:
vfn = get_value_paths(T)

fig, ax = plt.subplots(1, 1, figsize=(10, 4), dpi=120)
for leaf, vf in vfn.items():
    ax.plot(*zip(*vf), label=leaf, c="C0", alpha=0.24)

In [ ]:
T_sb = T
pos_sb = nx.nx_agraph.graphviz_layout(T_sb, prog="dot", args="")

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=300)
draw_tree(T_sb, pos_sb, ax)
fig.savefig("sb_tree.pdf")

Strong branching is provably the best branching rule in terms of search tree efficiency (the overall number of nodes). However, the exhaustive enumeration of all candidates coupled with invoking an expensive lp solver twice for each, makes the SB rule very computationally expensive in practice.

<br>

### Pseudocost branching

Whenever a fractional variable $j$ is picked for branching and both sub-porblems have feasible lp relaxations, we can compute its, so called, _branching pseudocosts_: $
    p^j_\pm = \frac{\Delta^j_\pm}{\phi^j_\pm}
$ where $
    \phi^j_- = \breve{x}^j - \lfloor \breve{x}^j \rfloor
$, and $
    \phi^j_+ = \lceil \breve{x}^j \rceil - \breve{x}^j
$. Conceptually, the $p^j_\pm$ measure the rate of degradation of the objective function value between the nested lp relaxations caused by splitting on a given variable. It is possible to view then as the local objective function sensitivities like $
    p^j_\pm
        \approx \partial_j (c^\top x)
$.

In [ ]:
from collections import namedtuple

Acc = namedtuple("Acc", "n,v")


def fractionality(x: ndarray) -> ndarray:
    """Compute the fractionality of each variable"""
    return np.stack((x - np.floor(x), np.ceil(x) - x), axis=-1)


def acc_get_estimate(acc: Acc, *, min: float = 1e-4, epsilon: float = 1e-5) -> ndarray:
    # compute the Laplace-corrected average estimate
    n = acc.n.sum(0, keepdims=True)
    coef = n / (n + len(acc.n) * epsilon)
    size = (acc.n + epsilon) * coef

    return np.clip(acc.v / size, min, None)

If the split results in an infeasible sub-problem, researchers suggest to use a [_fake objective value_](http://www.or.deis.unibo.it/andrea/pscost-ISMP2009.pdf), computed based on the parent's lp value and the averaged pseudocost of all other variables multiplied by $\phi^j_\pm$:
$$
\tilde{p}^j_\pm
    = \frac{
        \overbrace{
            \breve{f} + A \bigl( \bar{p}_\pm \phi^j_\pm + \epsilon \bigr)
        }^{\text{fake objective value}}
        - \breve{f}
    }{\phi^j_\pm}
    \,, $$
for a small $\epsilon > 0$, a large $A > 0$, and $\bar{p}_\pm$ -- the average pseudocost across the integer variables.

Whenever a pseudocost for a fractional variable is not available, we use partial strong branching to seed the initial up-lo gain estimates.

In [ ]:
from math import isfinite


def update_pseudocosts(pc: Acc, T: nx.DiGraph, node: int) -> None:
    p = T.nodes[node]["p"]

    # compute pseudocosts
    for c, dt in T[node].items():
        # decide which pseudocost to update
        k = 0 if dt["key"] < 0 else 1
        j, g, f = dt["j"], dt["g"], dt["f"]

        # compute pseudocosts and handle infeasible lp gains
        pcost = g / f
        if not isfinite(pcost):
            # on the one hand, we want the pcosts to reflect the dual
            #  gain from splitting by a variable, and at the same time
            #  cut off a sub-problem as quickly as possible. On the
            #  other hand, spoiling the pcosts now with a really HIGH
            #  gain, is bad for the using pcost estimate at other nodes.
            continue

            # fake = lp.fun + (avg_pcost * f + eps) * LARGE
            pc_avg = pc.v[: p.m].sum(0) / pc.n[: p.m].sum(0)

            # pcost = (fake - lp.fun) / f
            pcost = (float(pc_avg[k]) * f + 1e-2) * 1e4 / f

        # bump the branching counter and update averages in-place
        pc.n[j, k] += 1
        pc.v[j, k] += pcost
        # pc.v[j, k] += (pcost - pc.v[j, k]) / pc.n[j, k]

Let's try the pseudocost branching approach

In [ ]:
node = None
with tqdm(ncols=70) as pb:
    n_lpit = 0
    env = GeneratorEnv()

    last = node
    (T, node), fin = env.reset(branching(p, nodesel_dfs_lb))

    # get the up-lo pseudocosts
    n = T.graph["p"].n
    pc = Acc(np.zeros((n, 2)), np.zeros((n, 2)))
    while not fin:
        pb.update(1)
        assert not np.isnan(pc.v).any()

        # get the up-lo branching gains
        p_, lp_, mask = sub_problem(T, node)
        cands = np.flatnonzero(mask)
        assert len(cands) > 0

        frac = fractionality(lp_.x)

        # pick which pseudocosts need to be initialized
        mask = ((pc.n == 0.0).any(-1)) & (mask > 0)
        # pb.set_postfix_str(f"{mask.sum()} {bnb.bnb.gap(T):.2%}")  # XXX slow
        if mask.any():
            gains, nit = get_lp_gains(p_, lp_, mask)
            n_lpit += nit

            # get the pseudocosts (costs as in `c_j`)
            pcost = np.clip(gains / frac, 0, abs(p_.c).max())
            pc.v[mask] = pcost[mask]
            pc.n[mask] = 10  # XXX we can tweak this parameter

        # decide, which variable to branch on
        gains = acc_get_estimate(pc, min=1e-5) * frac
        # scores = mu * gains.max(-1) + (1 - mu) * gains.min(-1)
        scores = gains[:, 0] * gains[:, 1]
        var = cands[scores[cands].argmax()]

        # branch
        last = node
        (T, node), fin = env.step(var)
        update_pseudocosts(pc, T, last)

assert node is None

Plot the primal-dual bounds evolution, the value function paths, and the search tree for pseudocost branching

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots(1, 1, figsize=(5, 2), dpi=300)
nn_pc, pv_pc, dv = map(np.array, zip(*T.graph["track"]))
ax.plot(dv)
ax.plot(pv_pc)
ax.axhline(sol.fun, zorder=-10, c="k", alpha=0.25)

In [ ]:
vfn = get_value_paths(T)

fig, ax = plt.subplots(1, 1, figsize=(10, 4), dpi=120)
for leaf, vf in vfn.items():
    ax.plot(*zip(*vf), label=leaf, c="C0", alpha=0.24)

In [ ]:
T_pc = T
pos_pc = nx.nx_agraph.graphviz_layout(T_pc, prog="dot", args="")

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=300)
draw_tree(T_pc, pos_pc, ax)
fig.savefig("pc_tree.pdf")

Use plotly

In [ ]:
draw_tree_plolty(
    T_pc,
    pos_pc,
    nodelist=None,
    labels=node_nfo(T_pc, nn_pc),
).show()

Plot the final pseudocost estimate

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 2), dpi=300)

lo, up = acc_get_estimate(pc).T
ax.plot(up / up.max(), label="up")
ax.plot(-lo / lo.max(), label="lo")
ax.legend(fontsize="xx-small")

<br>

### SCIP

We've got our own custom [build](https://github.com/ivannz/ecole) of ecole.

In [ ]:
# import os, sys

# sys.path.append(
#     os.path.abspath("../../../repos_with_rl/copt/ecole/build/cmake/python/ecole")
# )

import ecole as ec

Let's use the tree tracer for SCIP
* TracedBranching env subclass appears to be less elegant, than putting the tracing logic in a reward or information function

In [ ]:
from ecole.environment import Branching
from toybnb.scip.tracer import NegLogTreeSize


scip_params = {
    "branching/scorefunc": "p",
    "randomization/permuteconss": False,
    "randomization/lpseed": 27372012,
    "branching/random/seed": 41,
    "branching/relpscost/startrandseed": 5,
}

rew_tracer = NegLogTreeSize()

env = Branching(
    observation_function=(
        ec.observation.NodeBipartite(),
        ec.observation.StrongBranchingScores(),
    ),
    information_function=rew_tracer,
    scip_params=scip_params,
)

Convert toy's MILP into SCIP's model

In [ ]:
fin = True
it = ec.instance.CombinatorialAuctionGenerator()
while fin:
    mod = ec.scip.Model.from_pyscipopt(to_scip(p))
    # mod = next(it)
    mod.disable_presolve()
    obs, act, rew, fin, nfo = env.reset(mod)
    break

assert not fin

Replicate the way [retro](https://arxiv.org/abs/2205.14345) decides fathomed nodes. We argue that it misrepresents the subset of fathomed nodes

In [ ]:
from toybnb.scip.tracer import Tracer

# https://github.com/scipopt/PySCIPOpt/blob/master/tests/test_tree.py#L42-L43
# https://github.com/mattmilten/TreeD/blob/main/src/treed/treed.py#L12

fathomed = set()


def retro(self: Tracer) -> None:
    """Detecting fathomed nodes as in https://arxiv.org/abs/2205.14345"""
    if len(self.trace_) < 2:
        return

    # bnb finished, check the last focus
    if self.focus_ is None:
        last, *_ = self.trace_[-1]
        if not self.T[last]:
            fathomed.add(last)
        return

    # the root may be visited twice, so we should not mark it as fathomed
    last, *_ = self.trace_[-2]
    if not self.T[last] and self.focus_ != last:
        fathomed.add(last)

SCIP is so much better than the thing we implemented, so there is little hope to see really deep trees on the problems, that toy can handle.

SCIP beats the toy even with random branching.

In [ ]:
from toybnb.scip.scip import from_scip_lp

db = []
fathomed.clear()
with tqdm(ncols=70) as pb:
    rng = default_rng()

    obs, act, rew, fin, nfo = env.reset(mod)
    m = env.model.as_pyscipopt()
    retro(rew_tracer.tracer)
    while not fin:
        pb.update(1)
        bi, sb = obs
        db.append(m.getDualbound())

        # read the local MILP at the focus node
        # x = from_scip_lp(env.model.as_pyscipopt())

        # pick a random index
        var = rng.choice(act)  # obs[act], act

        # or use sb scores
        var = act[sb[act].argmax(-1)]
        # branch
        obs, act, rew, fin, nfo = env.step(var)
        retro(rew_tracer.tracer)

        m = env.model.as_pyscipopt()

In [ ]:
from matplotlib import pyplot as plt

fig, ax = plt.subplots(1, 1, dpi=300)
nn_scip, pv_scip, dv = map(np.array, zip(*rew_tracer.tracer.trace_))
ax.plot(dv)
ax.plot(pv_scip)
ax.plot(db)
ax.axhline(sol.fun, zorder=-10, c="k", alpha=0.25)

In [ ]:
T_scip = rew_tracer.tracer.T.copy()
pos_scip = nx.nx_agraph.graphviz_layout(T_scip, prog="dot", args="")

In [ ]:
lab = dict(zip(nn_scip, range(len(nn_scip))))

In [ ]:
st = nx.get_node_attributes(T_scip, "status")
labels = {n: lab.get(n) for n in pos_scip if n in lab}  #  if st[n] == Status.CLOSED}

for n in fathomed:
    T_scip.nodes[n]["status"] = -999

In [ ]:
fig, ax = plt.subplots(1, 1, dpi=300)
draw_tree(T_scip, pos_scip, ax, nodelist=list(rew_tracer.tracer.fathomed_ | fathomed))

artists = nx.draw_networkx_labels(T_scip, pos_scip, ax=ax, font_size=4, labels=labels)
for t in artists.values():
    t.set_zorder(10)
fig.savefig("scip_tree.pdf")

Use plotly

How fast does each method decrease the primal bound?

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 2), dpi=300)

ax.semilogx(pv_rng, label="rng", alpha=0.75)
ax.semilogx(pv_rng_dfs, label="rng-dfs", alpha=0.75)
ax.semilogx(pv_sb, label="sb", alpha=0.75)
ax.semilogx(pv_pc, label="pc", alpha=0.75)

ax.semilogx(pv_scip, label="scip", alpha=0.75)

ax.axhline(sol.fun, zorder=-10, c="k", alpha=0.25)
ax.legend(fontsize="xx-small", ncol=4)

<br>

In [ ]:
n = nn_scip[-1]  # nn_scip[264], nn_scip[267]
while True:
    print(T_scip[n])
    for n in T_scip.pred[n]:
        break

    else:
        break

In [ ]:
list(map(len, (T_rng, T_rng_dfs, T_sb, T_pc, T_scip)))

<br>

<br>

In [ ]:
assert False

<br>

# Log

Reinventing the bicycle gives one better understanding of its inner workings and design.
So in this notebook I re-implement a BnB algorithm for Mixed Integer Linear Programs.

The following log is related to an earlier version and development of the toy bnb algorithm around the 8th of October, 2022.

When doing this study i proceeded through the following steps:
1. i started with defining a MILP specification format, that compatible with scipy's `linprog` API
  * initially i used list of tuples format for lower/upper bounds, but then, after digging through the code of `scipy.optimize` i discovered that passing $n \times 2$ numpy arrays with $\pm\infty$ works just fine (and is preferable, actually).

2. then i implemented the key feasibility checkers
  * bounding box $x_j \in \bigl[l_j, u_j\bigr]$
  * upper bound linear inequality constraints $A_\mathrm{ub} x \leq b_\mathrm{ub}$
  * linear equality constraints $A_\mathrm{eq} x = b_\mathrm{eq}$
  * integer feasibility $x \in \mathbb{Z}^m \times \mathbb{R}^{n-m}$

3. the `lpsolve` procedure, at first, dealt with list-of-tuples bounds, but was later reduced to an interface function

4. then `new` and `add` functions were added. Initially, the lp relaxation was computed outside of `add`, but it was reasonable to put the step inside

5. then i implemented the `bnb_begin` function (used to call `lpsolve`, before moving into `add`) and sketched the main loop of the bnb search: pruning by lower bound, picking the next node to process, and splitting (no infeasibility/integer feasibility fathoming yet). There i immediately started using the min-heap for pruning certifiably sub-optimal sub-problems.

6. with adding incumbent tracking, wild random branching, and variable splitting blocks the loop's body grew ever larger. So i factored three procedures out of it: `bnb_prune`, `bnb_update_incumbent`, and `bnb_branch`.
  - atm, we use a single global incumbent, but likely that the MILP has many solutions
    - [ ] implement storage for the case when there is solution multiplicity (18th of November, 2022)
  - [x] it would be structurally nice to make the stats more `recursive`, and let each node have its own set of incumbents that are best-so-far in that node's sub-problem.

7. variable picking logic was then also moved outside of the branching routine

8. i tinkered with a method to conveniently store branching rules' data and implemented node-local dunder attributes, that allowed the rules to be stateful
  - [ ] add a mask that indicates which variables were branched on, so that if we ever want to mix up branching rules on the fly, that they can communicate each other's choices

9. initially my bnb implementation __did not re-introduce__ not yet fully processed nodes into the dual bound max-heap, which resulted in sub-optimal solutions, since __not all branching alternatives were explored__.
  - [x] currently the dual bound max-heap also serves as the sub-problem prioritizer. This has to change.
  - this is the reason i decided to study bnb by _reinventing_ it: it was not clear to me, based on my tinkering with `ecole` and tree search retrieval by RETRO, how SCIP explored branching alternatives, if at all. Also i had a misconception about what gap it uses to gauge optimization progress.

10. after many attempts to track the status of each node (global fathomed/pruned sets, local flags) it finally dawned on me to introduce common node status codes, that allowed finer tracking of fathomed nodes:
  - __INFEASIBLE__: lp relaxation is infeasible, hence so the node's sub-problem is also infeasible
  - __INTEGER FEASIBLE__: the relaxation produced a solution that satisfies the integrality constraints, hence a global optimum for the node's MILP sub-problem was found, and there is no need to process it further
  - __PRUNED__: the lp produced a lower bound on all possible solutions to this node's MILP, that is higher than some integer feasible solution founds elsewhere. Thus we have a theoretical guarantee that it cannot contain a solution to the original root MILP.
  - __CLOSED__ and __OPEN__: fully processed nodes or nodes that still have some un-branched fractional variables, respectively.
  - status code greatly helped with debugging the search and understanding if it was operating correctly

11. I added gap tracking, `bnb_scip_gap` as defined by SCIP, to monitor progress, and finally put together the `bnb_search` procedure from the bare loop, that was running in a cell

12. (around 15th of October) Having become content with the slow but steady operation on the toy problems, i decided to apply this code on the crab allocation problem. This attempt failed miserably, since the algorithm worked way to slow. Even with the problem $56$ times smaller, the speed was still a huge issue. And the `linprog` solver was very unhappy with the allocation problem, complaining about its ill-posedness.
  - Also strong branching preformed much worse than random branching, for some reason. I suspect the issue is with incorrect node prioritization (indeed it was in hindsight)

13. I started experimenting with `presolve` in `scipy.optimize` sub-package, the code which i studied for hints at ways to improve the runtime. Serendipitously, i stumbled on the HiGHS linear solver, which tremendously sped up the procedure. Now at least the crab problem's gap started decreasing!

14. I suspected that backtracking to a still open node and branching on another variable and then diving could potentially produce a sub-problem the lp feasibility of which is covered by the feasibility region of some other __already__ solved problem. I confirmed by suspicions by inspecting the bounding boxes of the sub-problems produced by the bnb.
  - [ ] ~~we need to implement a lookup for solved sub-MILP that matches by covering bounding boxes (see `bnb_find_solved` stub and `bnb_update_interval_trees`)~~
  - (18th of November, 2022) the problem of feasibility subset revisits was due to branching more than once from every open node (see below, and the newer description above)

15. Started taking node scheduling responsibility off the dual bound max-heap
  - [x] `bnb_schedule_node`, `bnb_select_node`
  - it used to be that the `duals` max heap was automatically purged at the end of the bnb loop (removing FEASIBLE nodes by updating the incumbent and never rescheduling nodes, that were CLOSED)
  
16. testing feasibility by computing slacks $\max\{Ax - b, 0\}$ to _zero_ is unnecessarily slow

17. migrate the code into a python package

18. change the order of node and branch selection calls, so as to make the search code more modular

19. realize that there is NEVER a need to revisit an earlier branched node: the excluded region cannot contain an integer-feasible solution, even under splits by other variables!
20. implemented control inversion (via python threading) for easier experimentation (around the 10th of November, 2022)


##### References
* a well written master thesis on learning to branch [Scavuzzo Montana (2020)](https://repository.tudelft.nl/islandora/object/uuid:e1c09189-0b8f-470f-be99-1e1cf04f805e)
* the phd thesis of the core developer of SCIP [Achterberg (2007)](https://depositonce.tu-berlin.de/items/9f46a10e-2f7b-4dea-8e27-9cae07de5258/full)
* the original paper that proposes BnB [Benichou et al. (1979)](https://doi.org/10.1007/BF01584074)

<br>

### on the lp solver backend

`linprog` of `scipy.optimize` automatically presolves the problem and resolves redundancies.
- we could use it to simplify the search
- the parent-child sub-problems be nested __geometrically__, which __does not entail algebraic__ similarity

```python
from copy import deepcopy
from scipy.optimize._linprog_util import _presolve, _postsolve, _LPProblem
    
lp = _LPProblem(
    p.c, p.A_ub, p.b_ub, p.A_eq, p.b_eq, p.bounds, None
)
lp_o = deepcopy(lp)

# c0 is a constant term in the objective after presolve
lp, c0, x, undo, complete, status, message = _presolve(
    lp, True, None, tol=1e-9
)

x, fun, slack, con = _postsolve(
    x, (lp_o._replace(bounds=lp.bounds), undo, 1, 1), complete
)
```

When the solver is "highs" `linprog` from `scipy.optimize` passes the parsed (`_parse_linprog`), but unmodified problem directly to `_linprog_highs`.
* it appears that `_getAbc` and other problem transformations are currently undergoing a deprecation cycle

`_linprog_highs` transforms the problem from $
    A_\mathrm{ub} x \leq b_\mathrm{ub}
$ to $
    -\infty \leq A_\mathrm{ub} x \leq b_\mathrm{ub}
$ and $
    A_\mathrm{eq} x = b_\mathrm{eq}
$ to $
    b_\mathrm{eq} \leq A_\mathrm{eq} x \leq b_\mathrm{eq}
$.

```python
from scipy.optimize._linprog_highs import _linprog_highs, _highs_wrapper
```

<br>

# Trunk

The bipartite observations from ecole

In [ ]:
# 0 obj 1:4 type onehot 5 lb 6 ub 7 reduced cost
# 8 lp-val 9 lp-frac 10 at lb 11 at ub 12 age
# 13 incumbent-val 14 avg-incumbent-val 15:19 is_basis onehot
bi.variable_features[0]

In [ ]:
shape = len(bi.row_features), len(bi.variable_features)
tab = sp.coo_array((bi.edge_features.values, bi.edge_features.indices), shape)
bi.row_features.shape
bi.variable_features[0]

In [ ]:
tab

In [ ]:
plt.imshow(tab.toarray())

In [ ]:
cos = tab.dot(tab.T).toarray()
plt.imshow(abs(cos))

In [ ]:
from scipy import sparse as sp

if "x" in globals() and isinstance(x, MILP):
    A_ub = x.A_ub
    n = sp.linalg.norm(A_ub, axis=-1, ord=2)
    U = sp.diags(np.where(n > 0, 1 / n, 1.0)) * A_ub
    cos = U.dot(U.T).toarray()

    plt.imshow(abs(cos))

<br>

### Stepping through Ecole and the toy

A helper

In [ ]:
def sp_csr_isclose(a, b):
    assert sp.isspmatrix_csr(a) and sp.isspmatrix_csr(b)
    return (
        np.all(a.indptr == b.indptr)
        and np.all(a.indices == b.indices)
        and np.allclose(a.data, b.data)
    )


def milp_isclose(a: MILP, b: MILP) -> bool:
    return (
        sp_csr_isclose(a.A_ub, b.A_ub)
        and np.allclose(a.b_ub, b.b_ub)
        and sp_csr_isclose(a.A_eq, b.A_eq)
        and np.allclose(a.b_eq, b.b_eq)
        and np.allclose(a.c, b.c)
    )

The ecole env and our env diverge

In [ ]:
from pyscipopt.scip import Model, Variable
from toybnb.scip.scip import from_scip, from_scip_lp

mod = ec.scip.Model.from_pyscipopt(to_scip(p))
mod.disable_presolve()
mod.disable_cuts()
mod.transform_prob()

(bi, obs), act, rew, fin, nfo = env.reset(mod)

x = from_scip_lp(env.model.as_pyscipopt())
it = branching(x)  # x = it.gi_frame.f_locals["p"]


n_it = 0
G, node = it.send(None)
while True:
    n_it += 1

    # Why, although scip's and toy's branching candidates have
    #  diverged, do their LP problems still coincide?
    m: Model = env.model.as_pyscipopt()
    x0, x = x, from_scip_lp(m)
    assert milp_isclose(x, x0)

    # SCIP shuffles variables around, so we need a theirs- to
    #  ours- alignment map
    # XXX o2t: O \to T, maps domain O into co-domain T
    d_o = sorted(m.getVars(True), key=Variable.getIndex)
    o2t = np.array([v.getCol().getLPPos() for v in d_o])
    t2o = o2t.argsort()

    # ours: branch based on the sb score
    dt = T.nodes[node]

    mask = np.unpackbits(dt["mask"], count=dt["p"].n, bitorder="little")
    cands = np.flatnonzero(mask)
    gains, _ = get_lp_gains(dt["p"], dt["lp"], mask)
    scores = gains[:, 0] * gains[:, 1]

    if not (set(act) <= set(o2t[cands])):
        break

    if not np.allclose(scores, obs[o2t], equal_nan=True):
        break

    var = np.nanargmax(scores)

    (bi, obs), act, rew, fin, nfo = env.step(o2t[var])
    G, node = it.send(var)

In [ ]:
print(n_it)
print(set(act) - set(o2t[cands]), set(o2t[cands]) - set(act))
print(obs[o2t[cands]])
print(scores[cands])
print(obs[act])
print(scores[t2o[act]])

In [ ]:
x.bounds

In [ ]:
ecx = env.tracer.T.nodes[env.tracer.focus_]["lp"].x

In [ ]:
[ecx[v.name] for v in d_o]

A proto hybrid nodesel

In [ ]:
from heapq import heappush, heappop, heapify
from toybnb.tree import DualBound


def least_dual_nodesel_init(G: nx.DiGraph) -> None:
    pq, dt = G.graph["queue"], G.nodes

    pq[:] = [DualBound(dt[n]["lp"].fun, n) for n in pq]
    heapify(pq)


def least_dual_nodesel(G: nx.DiGraph, *reschedule: int) -> int:
    pq, dt = G.graph["queue"], G.nodes
    for n in reschedule:
        if dt[n]["status"] == Status.OPEN:
            heappush(pq, DualBound(dt[n]["lp"].fun, n))

    while pq:
        n = heappop(pq).node
        if dt[n]["status"] == Status.OPEN:
            return n

    raise IndexError


def nodesel(G: nx.DiGraph, *reschedule: int) -> int:
    if G.graph["incumbent"].x is None:
        return bnb.nodesel_dfs(G, *reschedule)

    if "_reheapified" not in G.graph:
        least_dual_nodesel_init(G)
        G.graph["_reheapified"] = True

    return least_dual_nodesel(G, *reschedule)

<br>

# Trunk

A helper function to find an existing node, the feasible region of
which is a superset of the given MILP, and the solution of which is
in the feasibility set of the MILP.

Indeed, if $A \subseteq B$ and $x_B \in A$, where $
    x_B \in \arg\min\{f(x) \colon x \in B\}
$, then $
    x_B \in \arg\min\{f(x) \colon x \in A\}
$, i.e. $x_B$ solves the problem $A$.

Hence, in order to reuse a previously solved problem $B$ for a problem $
    \min\{f(x) \colon x \in A\}
$ we need to check whether $
    x_B \in A \subseteq B
$.

Note, that the second term in the condition we use to find suitable
sub-problems is a simple point-set inclusion. Thus, we can
1. filter all MILPs whose lp solution `lp.x` is feasible in the current sub-problem `p.bounds`
2. among those, search for set-set inclusion using `is_feasible_box` (`.bounds`)

We're dealing with $B$ $m$-dimensional boxes of the form $
    \prod_{j=1}^m [a_{bj}, z_{bj}]
$ with $a_{bj} \leq z_{bj}$, $b \in B$. We wish to
1. quickly add a new box $b$ to the data structure
2. cheaply lookup all boxes $J \subseteq B$ such that $b \subseteq j$ for all $j \in B$

A rectangle is a set of the form $\prod_j A_j$, with $(A_j)_{j=1}^n$ given
by closed intervals, possibly unbounded. We need to implement an algorithm,
that efficiently finds at least one rectangle $R$ in a collection $
    \mathcal{R} = (R_k)_{k=1}^p
$, such that the rectangle $Q$ is a __subset__ of $R$.

<br>